In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Freelancer/Spam Campaign Detection')

In [4]:
import json
import os
import csv
import re
import pandas as pd
import numpy as np
import glob
from urllib.parse import urlparse
import requests
from collections import defaultdict, namedtuple
from Tree.node import FPNode
from Tree.tree import FPTree

In [5]:
def parse(path="C:/Users/aniru/Downloads/fraudulentDataset/dhruval"):
    full_data=[]
    tid=0
    all_files = os.listdir(path)
    
    features = ['readability', 'emailLayoutHtml', 'emailLayoutText', 
                'embeddedURLs', 'originDomain', 'originSourceIP', 
                'language', 'originEmailAddress', 'originName', 
                'subject', 'originDate', 'attachments']
    
    for file in all_files:
        if file.endswith("eml"):
            continue
        data=[]
        file_path = os.path.join(path, file)
        with open(file_path, 'r', encoding='utf8') as f:
            json_dict = json.load(f)
        for feature in features:
            
            if feature == 'readability':
                temp = json_dict.get(feature, "None")
                if isinstance(temp, dict):
                    content = temp.get('contentType', "None")
                    charset = temp.get('charset', "None")
                    data += ["{}:content".format(content),
                             "{}:char".format(charset)] 
                else:
                    data += ["{}:content".format("None"),
                             "{}:char".format("None")]
                continue
            
            if feature == 'embeddedURLs':
                urls = json_dict.get(feature, "None")
                if isinstance(urls, list):
                    for item in urls:
                        tokens =urlparse(item['url'])
                        data+=['{}:url_netloc'.format(tokens.netloc)]
                        if tokens.path and not tokens.path == '/' :
                            data+=['{}:url_path'.format(tokens.path)]
                        if tokens.params:
                            data+=['{}:url_params'.format(tokens.params)]
                else:
                    data += ['None:url_netloc',
                             'None:url_path',
                             'None:url_params']
                continue
            
            # if feature == 'top10simpleWord':
            #     words = json_dict.get(feature, "None")
            #     if not isinstance(words,dict):
            #         for item in words.keys():
            #             data += ['{}:word'.format(item)]
            #     else:
            #         data += ["None:word"]
            #     continue

            if feature == 'attachments':
                attached = json_dict.get(feature, "None")
                if not isinstance(attached, list):
                    for attachment in attached:
                        filename = attachment.get('fileName', "None")
                        data+=['{}:attachment'.format(filename)]
                else:
                    data+= ['None:attachment']
                continue
            # All other features are handled there other than mentioned and checked above
            val = json_dict.get(feature, "None")
            data += ["{}:{}".format(val, feature)]
        tid+=1
        data+=[str(tid)+':id']
        full_data.append(data)      
        
    return full_data

# New Section

In [6]:
def defdict(dicts):
    return defaultdict(lambda: 0, dicts)

def find_frequent_itemsets(transactions, minimum_support, params, include_support=True):
    """
    Find frequent itemsets in the given transactions using FP-growth. This
    function returns a generator instead of an eagerly-populated list of items.
    The `transactions` parameter can be any iterable of iterables of items.
    `minimum_support` should be an integer specifying the minimum number of
    occurrences of an itemset for it to be accepted.
    Each item must be hashable (i.e., it must be valid as a member of a
    dictionary or a set).
    If `include_support` is true, yield (itemset, support) pairs instead of
    just the itemsets.
    """
    if 0 < minimum_support <= 1:
        minimum_support = minimum_support * len(transactions)
    
    onflytran=transactions[len(transactions)//2:]
    transactions=transactions[:len(transactions)//2]
    items = defaultdict(lambda: 0)  # mapping from items to their supports

    # if using support rate instead of support count
    

    # Load the passed-in transactions and count the support that individual
    # items have.
    for transaction in transactions:
        for item in transaction:
            items[item] += 1
    

    
    # items = dict(
    #     (item, support) for item, support in items.items() if support >= minimum_support
    # )
    # Remove infrequent items from the item support dictionary and
    #sorts the dictionary  according to value
    items= defdict({k: v for k, v in 
                   sorted(items.items(), 
                          key=lambda item: item[1], 
                          reverse=True) \
                   if v >= minimum_support or \
                   k.split(":")[1] == 'id'or \
                   k.split(":")[1] == 'content'})
    # Build our FP-tree. Before any transactions can be added to the tree, they
    # must be stripped of infrequent items and their surviving items must be
    # sorted in decreasing order of frequency.
    
    master = FPTree(items)
    print(master.inspect())
    master.add(transactions,on_fly = False)
    # Tree is built

    # while True:
    #     if is_new_transcation:
    #         master.add(onflytran, on_fly = True)

    #         # On fly method:
    #         append to header_table
    #         resort it
    #         key: value   difference key: value 
    # print(master.inspect())
    return master.getcampaign(params)

In [7]:
dataset = parse("/content/drive/My Drive/Freelancer/Spam Campaign Detection/dhruval/")
dataet = dataset * 10

In [9]:
len(dataset)

16

In [15]:
dataset = dataset * 10

In [16]:
param=[1,1,[],1]
frequent_items = find_frequent_itemsets(dataset,2,param)
print(f"No of camp. is {len(frequent_items)}")
# for aaa in aa: print(aaa)

Tree:
  <FPNode (root)>

Routes:
None
/help/community/png-priv.html:url_path
/help/policies/hub.html:url_path
None:url_params
iso-8859-1:char
multipart/alternative:content
T:emailLayoutText
None:url_params
iso-8859-1:char
None:url_params
None:url_params
No of camp. is 10


In [14]:
d={'click3.ebay.com:7': 28, 'pages.ebay.com:7': 25, 'iso-8859-1:3': 12, 'text/html:1': 7, 'en:2': 7, 'T:6': 6, 'www.ncua.gov:7': 5, 'multipart/alternative:1': 5, 'TNTNTNTNNUTNTUTUTNNUTUTUTUTUTUTUTUNTU:5': 2, 'cgi4.ebay.com:7': 2, '/ws1/eBayISAPI.dll:8': 2, '/acounts/memb/avncenter/dll87443/.BayISAPI.dll/hgdas676bsda6gwcv7zfcwfcwf34gfwf23g235f134f3fg3f&bhdfahva68532hbhwseBayISAPI.dllPaymentLanding&ssPageName=hhpayUSf&=userhgads&secure&ssl7r2vbd7d5b.html:8': 2, '/help/policies/privacy-policy.html:8': 2, '/help/policies/user-agreement.html:8': 2, '/community/aboutebay/:8': 2, 'www2.ebay.com:7': 2, '/aw/marketing.shtml:8': 2, '/securitycenter/:8': 2, '/help/policies/hub.html:8': 2, '/sitemap.html:8': 2, '/help/myebay/index.html:8': 2, '/help/community/png-priv.html:8': 2, 'your eBay account could be suspended due updates.:4': 2, 'System maintenance: update your Federal Credit Union:4': 2, '9:0': 1, '10:0': 1, '11:0': 1, '12:0': 1, '13:0': 1, '14:0': 1, '15:0': 1, '16:0': 1, 'youe eBay account could be suspended due Updates.:4': 1, '62.193.219.73:7': 1, '1:0': 1, '62.193.210.146:7': 1, '2:0': 1, '3:0': 1, '4:0': 1, '5:0': 1, 'windows-1251:3': 1, 'eBay - verify your account information:4': 1, '6:0': 1, 'PayPal Flagged Account:4': 1, 'TNNTNNTNNNNTNNTNNNUNNNTNNTNNTNNTNTNTNNT:5': 1, 'www.idhp.com.br:7': 1, '/modules/FCKeditor/upload/Media/abuse:8': 1, 'www.paypal.com:7': 1, '/cgi-bin/webscr:8': 1, '7:0': 1, '***Urgent Fraud Prevention Group Notice***:4': 1, '8:0': 1}

In [9]:
class c:
    def __init__(self,a):
        self._a={0:a}
    @property
    def a(self):
        return self._a

C=c(5)
C.a[0]+=1
print(C.a)

{0: 6}


In [10]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [11]:
simpDat = loadSimpDat()